<a href="https://colab.research.google.com/github/yogeshtak/Learning/blob/master/Topic_model_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv('Comments1.csv')
data.dropna(subset=['comment1'], inplace=True)

In [ ]:
data.head()

,Response_ID,comment1,comment2
0,cf561c54-f8e3-cd65-7a71-2b7d8e00078d,selling more product,having enough sales to get through
1,e95e599f-2ff9-8751-72ba-6e75abaac643,More employee retention.,None at all.
2,e1959062-e65a-4b7b-b947-a7be889d0b9b,customer insights,COVID 19
3,5f11e0f5-4820-3cf8-3dd0-d73ae2f0b10c,more customers,Keeping all the employees safe
4,13a6537f-c7ea-f76a-8a5a-648d2abd4ea8,Everything,People getting sick


In [ ]:
data_comment5 = data[['comment1']]
data_comment5['index'] = data_comment5.index
data5 = data_comment5

print(len(data5))
print(data5[:5])

217
                   comment1  index
0     selling more product       0
1  More employee retention.      1
2         customer insights      2
3           more customers       3
4               Everything       4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
def lemmatize_stemming(text):
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
      result.append(lemmatize_stemming(token))
  return result



In [ ]:
comment_sample = data5[data5['index']==156].values[0][0]

print('Original Text: ')
words = []
for word in comment_sample.split(' '):
  words.append(word)
print(words)
print('\n\n tokenized and lemmatized comment: ')
print(preprocess(comment_sample))

Original Text: 
['I', 'would', 'be.very', 'impressed', 'cause', 'I', 'worked', 'hard', 'for', 'it\n']


 tokenized and lemmatized comment: 
['impress', 'caus', 'work', 'hard']


In [ ]:
processed_comment5 = data5['comment1'].map(preprocess)
processed_comment5[:10]

0                                      [sell, product]
1                                    [employe, retent]
2                                    [custom, insight]
3                                             [custom]
4                                                   []
5    [think, help, higher, compani, actual, listen,...
6    [think, have, peopl, avail, interact, custom, ...
7                            [avail, comprehens, tool]
8                      [know, work, safe, work, place]
9            [great, great, great, good, night, sleep]
Name: comment1, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_comment5)

In [ ]:
count = 0

for k,v in dictionary.iteritems():
  print(k,v)
  count+=1
  if count > 10:
    break

0 product
1 sell
2 employe
3 retent
4 custom
5 insight
6 actual
7 appeas
8 compani
9 donor
10 help


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_comment5]
bow_corpus[11]

[(1, 1), (3, 1)]

In [ ]:
bow_doc_11 = bow_corpus[11]

for i in range(len(bow_doc_11)):
  print("Word{} (\"{}\") appears {} time.".format(bow_doc_11[i][0], dictionary[bow_doc_11[i][0]], bow_doc_11[i][1]))

Word1 ("insight") appears 1 time.
Word3 ("better") appears 1 time.


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
from pprint import pprint

for comment in corpus_tfidf:
  pprint(comment)
  break

[]


In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.950*"feedback" + 0.012*"custom" + 0.012*"better" + 0.012*"collect" + 0.012*"insight"
Topic: 1 
Words: 0.534*"better" + 0.405*"insight" + 0.020*"custom" + 0.020*"feedback" + 0.020*"collect"
Topic: 2 
Words: 0.372*"insight" + 0.355*"custom" + 0.212*"collect" + 0.058*"better" + 0.003*"feedback"
Topic: 3 
Words: 0.873*"better" + 0.082*"collect" + 0.031*"insight" + 0.007*"custom" + 0.007*"feedback"
Topic: 4 
Words: 0.802*"custom" + 0.115*"insight" + 0.057*"better" + 0.021*"collect" + 0.005*"feedback"
Topic: 5 
Words: 0.439*"collect" + 0.356*"feedback" + 0.131*"better" + 0.065*"insight" + 0.010*"custom"
Topic: 6 
Words: 0.833*"custom" + 0.129*"insight" + 0.025*"collect" + 0.006*"better" + 0.006*"feedback"
Topic: 7 
Words: 0.536*"custom" + 0.190*"feedback" + 0.186*"insight" + 0.044*"better" + 0.044*"collect"
Topic: 8 
Words: 0.339*"custom" + 0.295*"insight" + 0.295*"feedback" + 0.058*"better" + 0.014*"collect"
Topic: 9 
Words: 0.426*"collect" + 0.252*"custom" + 0.217*"feedb

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.967*"custom" + 0.019*"better" + 0.005*"feedback" + 0.005*"collect" + 0.005*"insight"
Topic: 1 Word: 0.513*"insight" + 0.247*"custom" + 0.153*"better" + 0.044*"feedback" + 0.043*"collect"
Topic: 2 Word: 0.367*"custom" + 0.314*"feedback" + 0.107*"better" + 0.107*"collect" + 0.107*"insight"
Topic: 3 Word: 0.413*"collect" + 0.336*"insight" + 0.203*"custom" + 0.038*"better" + 0.010*"feedback"
Topic: 4 Word: 0.472*"insight" + 0.218*"better" + 0.149*"feedback" + 0.112*"custom" + 0.048*"collect"
Topic: 5 Word: 0.330*"insight" + 0.326*"feedback" + 0.263*"custom" + 0.067*"collect" + 0.014*"better"
Topic: 6 Word: 0.915*"better" + 0.046*"custom" + 0.025*"feedback" + 0.007*"collect" + 0.007*"insight"
Topic: 7 Word: 0.769*"feedback" + 0.168*"custom" + 0.044*"better" + 0.009*"insight" + 0.009*"collect"
Topic: 8 Word: 0.640*"custom" + 0.143*"insight" + 0.099*"feedback" + 0.086*"collect" + 0.032*"better"
Topic: 9 Word: 0.649*"collect" + 0.184*"feedback" + 0.099*"custom" + 0.059*"insigh